In [1]:
##Imports

import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import os
from datetime import date as dateClass, timedelta

In [2]:
## Declaring dictionaries for storage of data

locationDictionary={}
categoryDictionary={}
tagsDictionary={}

In [3]:
#Date Calculations

dateCorrect=False
while dateCorrect!=True:
    dateInput=raw_input("Enter a VALID date in MM-DD-YYYY format. Example: 01-31-2018 or LEAVE BLANK for DEFAULTof past 7 days\n")
    if(dateInput==''):
        print "No Date Provided. Using a default of past 7 days data."
        dateInput1=dateClass.today() - timedelta(days=7)
        year= dateInput1.year
        month= dateInput1.month
        date= dateInput1.day
        print "Starting to scrape newsroom data for analysis...."
        break
    else:
        month=int(dateInput.split("-")[0])
        date=int(dateInput.split("-")[1])
        year=int(dateInput.split("-")[2])
        try:
            dateInput1=dateClass(year,month,date)
            if (dateInput1 <= dateClass.today()):
                dateCorrect=True
                print "Starting to scrape newsroom data for analysis....\n"
            else:
                print "Date entered is after today's date. Enter today's date or past date\n"
        except ValueError:
            print "INVALID DATE entered. Try again..\n"


Enter a VALID date in MM-DD-YYYY format. Example: 01-31-2018 or LEAVE BLANK for DEFAULTof past 7 days

No Date Provided. Using a default of past 7 days data.
Starting to scrape newsroom data for analysis....


In [4]:
#Making a request to the webpage and getting beautiful soup object

page = requests.get("https://www.newswire.com/newsroom")    
if page.status_code==200:   
    soup = BeautifulSoup(page.content, 'html.parser')
else:
    print "HTTP Request Rejected by: https://www.newswire.com/newsroom\tTry again later"

In [5]:
#function to map months to month number
def monthToIntMonth(articleMonth):
    switcher = {
            "Jan":1,
            "Feb":2,
            "Mar":3,
            "Apr":4,
            "May":5,
            "Jun":6,
            "Jul":7,
            "Aug":8,
            "Sep":9,
            "Oct":10,
            "Nov":11,
            "Dec":12
        }
    return switcher.get(articleMonth, "Invalid month")

#function to generate soup object for inidivual article page
def getSoupArticle(url):
    pageArticle = requests.get(url)    
    if pageArticle.status_code==200:   
        soupArticle = BeautifulSoup(pageArticle.content, 'html.parser')
    else:
        print "HTTP Request Rejected by: "+url+"\tTry again later"
    return soupArticle

#function to extract location, category and tag data from article URLs
def getArticleData(soupArticle):
    #getting location data
    locList=soupArticle.select("p strong.date-line.color-pr")[0].get_text().strip().lower().split(",")[:-2]
    location=[]
    for loc in locList:
        loc=loc.lstrip().rstrip()
        location.append(loc)
    for loc in location:     
        if loc in locationDictionary.keys():
            locationDictionary[loc]+=1
        else:
            locationDictionary[loc]=1
    
    #getting category data
    try:
        catSoup=soupArticle.select("p[class=mb-0]")[0]
        catListLen=len(catSoup.select("a"))
        for i in range(0,catListLen):
            catList=str(catSoup.select("a")[i].get_text().encode('utf-8').lower()).split(",")
            for cat in catList:
                cat=cat.lstrip()
                if cat in categoryDictionary.keys():
                    categoryDictionary[cat]+=1
                else:
                    categoryDictionary[cat]=1
    except IndexError:
        pass
    #getting tag data
    try:
        tagSoup=soupArticle.select("p[class=mb-0]")[1]
        tagListLen=len(tagSoup.select("a"))
        for i in range(0,tagListLen):
            tagList=str(tagSoup.select("a")[i].get_text().encode('utf-8').lower()).split(",")
            for tag in tagList:
                if tag in tagsDictionary.keys():
                    tagsDictionary[tag]+=1
                else:
                    tagsDictionary[tag]=1
    except IndexError:
        pass

In [6]:
#traversing articles using beautiful soup until the appropriate date
nextPageTraverse=True
while(nextPageTraverse):
    divs=soup.select("div.news-item-body")
    for div in divs:
        p=div.select("time.ln-date")[0]
        articleDate=p.get_text().split(" ")
        articleDay=int(str(articleDate[1]).replace(',',''))
        articleMonth=int(monthToIntMonth(str(articleDate[0]).strip()))
        articleYear=int(articleDate[2])
        if(articleYear>=year):
            if(articleMonth>=month):
                if(articleDay>=date):
                    url="https://www.newswire.com/"+div.select("a.content-link")[0]['href']
                    print str(articleMonth) + "/" + str(articleDay) + "/" + str(articleYear)+" -- "+url
                    soupArticle=getSoupArticle(url)
                    getArticleData(soupArticle)
                else:
                    nextPageTraverse=False
                    break
            else:
                nextPageTraverse=False
                break
        else:
            nextPageTraverse=False
            break
    nextPage=soup.select("div.chunkination.chunkination-centered ul")[2]
    try:
        nextPage=str(nextPage.find("a")['href'])
        url="https://www.newswire.com/"+nextPage
        page = requests.get(url)    
        if page.status_code==200:   
            soup = BeautifulSoup(page.content, 'html.parser') 
    except:
        print 'No Next Page. End of results'
        nextPageTraverse=False

8/27/2018 -- https://www.newswire.com/news/stunning-new-necklaces-that-awaken-the-warrior-princess-within-20617460
8/26/2018 -- https://www.newswire.com/news/millennial-mom-jenna-barnett-spotlights-female-founded-brands-in-20614837
8/26/2018 -- https://www.newswire.com/news/i-accident-lawyer-shares-tips-for-passengers-involved-in-an-uber-20613409
8/25/2018 -- https://www.newswire.com/news/warning-the-state-of-mexico-about-psychiatric-abuse-20615383
8/25/2018 -- https://www.newswire.com/news/the-western-way-highlights-leaders-from-across-the-eastern-plains-20614836
8/25/2018 -- https://www.newswire.com/news/peace-and-culture-gala-is-launching-its-first-edition-20610583
8/25/2018 -- https://www.newswire.com/news/lottery-apps-smart-numbers-score-big-wins-20614060
8/25/2018 -- https://www.newswire.com/news/sa-co-proudly-announces-launch-and-expansion-of-new-marketing-division-20615028
8/25/2018 -- https://www.newswire.com/news/ccr-circle-computer-resources-inc-appears-on-the-inc-5000-for-t

8/23/2018 -- https://www.newswire.com/news/learningrx-red-bank-student-with-adhd-wins-student-of-the-year-20567142
8/23/2018 -- https://www.newswire.com/news/house-cladding-and-renovation-expert-peter-bracey-helping-sydney-20610882
8/23/2018 -- https://www.newswire.com/news/mario-lopez-to-host-inaugural-classic-new-york-gala-bringing-together-20610913
8/23/2018 -- https://www.newswire.com/news/gen-paul-nakasone-and-gchq-director-jeremy-fleming-to-keynote-at-9th-20611484
8/23/2018 -- https://www.newswire.com/news/geistm-named-fastest-growing-martech-platform-with-three-year-revenue-20612008
8/23/2018 -- https://www.newswire.com/news/artekal-music-releases-artekalmoji-companion-app-20612069
8/23/2018 -- https://www.newswire.com/news/periscope-data-experiences-dramatic-growth-in-2018-develops-new-20611717
8/23/2018 -- https://www.newswire.com/news/collin-granger-named-ceo-and-president-of-mapcom-systems-20611783
8/23/2018 -- https://www.newswire.com/news/netsertive-named-an-inc-5000-faste

8/22/2018 -- https://www.newswire.com/news/pharmacy-podcast-network-ppn-announces-launch-into-internet-radio-20608881
8/22/2018 -- https://www.newswire.com/news/international-association-of-women-recognizes-tsahi-alexander-as-a-20610269
8/22/2018 -- https://www.newswire.com/news/telaeris-introduces-xpid-android-handheld-security-badge-reader-20610555
8/22/2018 -- https://www.newswire.com/news/alliance-of-comprehensive-planners-announces-featured-speakers-special-20610775
8/22/2018 -- https://www.newswire.com/news/discovery-senior-living-commemorates-groundbreaking-of-new-state-of-20610923
8/22/2018 -- https://www.newswire.com/news/taulia-achieves-record-q2-results-with-new-deals-and-6-billion-released-20610606
8/22/2018 -- https://www.newswire.com/news/tonys-picks-exhibits-track-record-of-winning-football-picks-in-new-site-20609620
8/22/2018 -- https://www.newswire.com/news/your-company-formations-provider-of-expert-company-formation-services-20609037
8/22/2018 -- https://www.newswire.

8/21/2018 -- https://www.newswire.com/news/spinal-singularity-receives-employment-achievement-award-from-triumph-20602858
8/21/2018 -- https://www.newswire.com/news/n8-identity-introduces-theaccesshub-plugin-for-zendesk-20609438
8/21/2018 -- https://www.newswire.com/news/co-inventor-of-blockchain-joins-celsius-network-20608765
8/21/2018 -- https://www.newswire.com/news/alive-magazine-to-be-featured-at-north-american-tough-mudder-events-20609021
8/21/2018 -- https://www.newswire.com/news/international-association-of-women-recognizes-cynthia-anderson-as-a-20603965
8/21/2018 -- https://www.newswire.com/news/international-association-of-women-recognizes-linda-steinkoetter-as-a-20600321
8/21/2018 -- https://www.newswire.com/news/yalbers-marketing-director-kobi-ben-meir-is-nominated-for-b2b-marketer-20609338
8/21/2018 -- https://www.newswire.com/news/berlin-packaging-opens-new-office-and-warehouse-in-illinois-to-support-20608865
8/21/2018 -- https://www.newswire.com/news/product-growth-leade

8/20/2018 -- https://www.newswire.com/news/blockchain-company-skycoin-looking-to-break-into-the-korean-market-20608838
8/20/2018 -- https://www.newswire.com/news/urban-pulse-launches-new-harvest-moon-collection-of-skin-care-and-20607120
8/20/2018 -- https://www.newswire.com/news/new-vr-gaming-studio-develops-software-to-successfully-surpass-6-500-20608083
8/20/2018 -- https://www.newswire.com/news/elumis-foundation-a-charity-formed-by-children-donates-1-000-solar-20605867
8/20/2018 -- https://www.newswire.com/news/verity-studios-drones-dance-with-drake-in-songs-elevate-and-look-alive-20597157
8/20/2018 -- https://www.newswire.com/news/the-composites-in-the-global-passenger-rail-market-is-expected-to-20605210
8/20/2018 -- https://www.newswire.com/news/thermaldyne-to-open-west-baton-rouge-plant-in-2018-20608829
8/20/2018 -- https://www.newswire.com/news/well-biotec-introduces-heat-treated-nano-lactobacillus-extracted-from-20577098
8/20/2018 -- https://www.newswire.com/news/officials-gath

In [7]:
#generating LocationAnalysis bar graph
print "\n\nLocation Analysis Graph (LocationAnalysis.png) is save locally in your CWD: "+os.getcwd()
fig, ax = plt.subplots(figsize=(50,25))
plt.bar(range(len(locationDictionary)), locationDictionary.values(), align='center')
plt.xticks(range(len(locationDictionary)), locationDictionary.keys(),rotation='vertical', fontsize='12')
plt.xlabel('Locations')
plt.ylabel('Number of Articles')
plt.savefig('LocationAnalysis.png')

printTable=""
while printTable=="":
    printTable=raw_input("\nDo you want to print the Location Analysis Table (Y/N)?: ")
    if(printTable=='Y' or printTable=='y'):
        #generating location analysis frequency table
        locDict1={}
        maxLen=0
        for k,v in locationDictionary.iteritems():
            k=k.encode('ascii','ignore')
            locDict1[k]=v
            if(maxLen<len(k)):
                maxLen=len(k)

        print "Location"+' '*(maxLen+2)+"| Number of Article"
        print '-'*(maxLen+32)
        for k,v in locDict1.iteritems():
            print k+' '*(maxLen-len(k)+10)+"|"+' '*10+str(v)
        break
    elif(printTable=='N' or printTable=='n'):
        break
    else:
        print "\nInvalid Input try again"
        printTable=""



Location Analysis Graph (LocationAnalysis.png) is save locally in your CWD: D:\HIGHER STUDIES\Stevens\Job Search\Fall Internship\Indeed\newswire.com\Coding Test

Do you want to print the Location Analysis Table (Y/N)?: N


In [8]:
#Generate CategoryAnalysis bar graph
print "\n\nCategory Analysis Graph (CategoryAnalysis.png) is save locally in your CWD: "+os.getcwd()
fig, ax = plt.subplots(figsize=(50,25))
plt.bar(range(len(categoryDictionary)), categoryDictionary.values(), align='center')
plt.xticks(range(len(categoryDictionary)), categoryDictionary.keys(),rotation='vertical', fontsize='12')
plt.xlabel('Categories')
plt.ylabel('Number of Articles')
plt.savefig('CategoryAnalysis.png')

printTable=""
while printTable=="":
    printTable=raw_input("\Do you want to print the Category Analysis Table (Y/N)?: ")
    if(printTable=='Y' or printTable=='y'):
        maxLen=0
        for k,v in categoryDictionary.iteritems():
            if(maxLen<len(k)):
                maxLen=len(k)

        print "Category"+' '*(maxLen+2)+"| Number of Article"
        print '-'*(maxLen+32)
        for k,v in categoryDictionary.iteritems():
            print k+' '*(maxLen-len(k)+10)+"|"+' '*10+str(v)
    elif(printTable=='N' or printTable=='n'):
        break
    else:
        print "\nInvalid Input try again"
        printTable=""



Category Analysis Graph (CategoryAnalysis.png) is save locally in your CWD: D:\HIGHER STUDIES\Stevens\Job Search\Fall Internship\Indeed\newswire.com\Coding Test
\Do you want to print the Category Analysis Table (Y/N)?: N


In [9]:
print "\n\nThe Tags are mostly unique for each article and thus plotting a bar graph wouldn't be feasible/readble. \
We thus only have a table for the same\n"

printTable=""
while printTable=="":
    print "\nThe Tags frequency table WILL BE QUITE LONG. PLEASE WAIT UNTIL IT IS COMPLETELY PRINTED, if opted to.."
    printTable=raw_input("\nDo you want to print the Tags Analysis Table (Y/N)?: ")
    if(printTable=='Y' or printTable=='y'):
        maxLen=0
        for k,v in tagsDictionary.iteritems():
            if(maxLen<len(k)):
                maxLen=len(k)

        print "Tags"+' '*(maxLen+6)+"| Number of Article"
        print '-'*(maxLen+32)
        for k,v in tagsDictionary.iteritems():
            print k+' '*(maxLen-len(k)+10)+"|"+' '*10+str(v)
    elif(printTable=='N' or printTable=='n'):
        break
    else:
        print "\nInvalid Input try again"
        printTable=""

print "\n\nPlease DONT FORGET to take a look at the LocationAnalysis and CategoryAnalysis plots saved locally in your CWD: "+os.getcwd()+"\n\n"



The Tags are mostly unique for each article and thus plotting a bar graph wouldn't be feasible/readble. We thus only have a table for the same


The Tags frequency table WILL BE QUITE LONG. PLEASE WAIT UNTIL IT IS COMPLETELY PRINTED, if opted to..

Do you want to print the Tags Analysis Table (Y/N)?: N


Please DONT FORGET to take a look at the LocationAnalysis and CategoryAnalysis plots saved locally in your CWD: D:\HIGHER STUDIES\Stevens\Job Search\Fall Internship\Indeed\newswire.com\Coding Test


